# API SNCF: Les retards de train

__Auteur :__ Steve Caron
__Date de creation :__ 28/01/2024

__Prerequis :__

* Python 3
* Une clef API SNCF stocké dans un environnement virtuel. Clé disponible sur : https://numerique.sncf.com/startup/api/
* Git

__Installation :__

* Cloner le projet

* Creer un environnement virtuel 

    ``py -m venv .venv``
    
* Activer l'environnement virtuel

    ``.venv/Scripts/activate``

* Installer les dépendance

    ``pip install -r requirements.txt``

__Params :__

* CLEF_API : nom sous lequel est enregistrer la cle API
* CODE_GARE : ID de la gare dont on veut récuperer les données

In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
import datetime

In [2]:
CLEF_API = "API_KEY"
CODE_GARE = "stop_area:SNCF:87713040"

Récupération de la clé API depuis l'environnement

In [3]:
load_dotenv()
api_key = os.getenv(CLEF_API)

Je récupère la date du jour et à partir de celle-ci creer les arguments de ma requete. 

In [4]:
aujourdhui = datetime.date.today()
hier_debut_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=0, minute=0 ,second=0)
hier_fin_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=23, minute=59 ,second=59)

Creation des fonctions pour coder et decoder les datetime

In [5]:
def convertir_en_string(dt):
    '''Cette fonction convertit un datetime en chaine de caractère'''
    return datetime.datetime.strftime(dt,'%Y%m%dT%H%M%S')

In [6]:
def convertir_en_datetime(str):
    '''Cette fonction convertit une chaine de caractère en datetime'''
    return datetime.datetime.strptime((str[:8]+str[9:]),'%Y%m%d%H%M%S')

In [7]:
def to_json(data,nom_fichier):
    '''Cette fonction permet d'enregistrer un fichier JSON'''
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [8]:
def requete_api(code_gare,code_reseau,date):

    base_url = "https://api.sncf.com/v1/coverage/sncf"
    #Requete sans le filtre sur les trains
    requete = f"{base_url}/stop_areas/{code_gare}/networks/{code_reseau}/arrivals?from_datetime={date}"
    reponse = requests.get(requete, auth=(api_key,""))
    reponse_json = reponse.json()
    to_json(reponse_json,"_test.json")
    
    return reponse_json

In [9]:
def requete_entre_dates(code_gare,code_reseau,date_min,date_max,liste_arrivee,liste_perturbation):

    derniere_requete = date_min

    print(f"Première requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau}")

    while derniere_requete < date_max:
        # Requete api
        reponse = requete_api(code_gare,code_reseau,derniere_requete)
        # extrait les arrivées
        arrivees = reponse["arrivals"]
        # extrait les perturbations
        perturbations = reponse["disruptions"]
        # Ajoute chaque arrivées à la liste
        [liste_arrivee.append(arrivee) for arrivee in arrivees]
        # Ajoute chaque perturbations à la liste
        [liste_perturbation.append(perturbation)  for perturbation in perturbations]
        # Si il a moins de 10 arrivées, alors la prochaine requete ne donnera rien, on retourne donc directement les résultats
        if len(arrivees) < 10:
            print(f"Dernière requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau} // dernieres requetes {len(arrivees)} arrivées")
            return liste_arrivee,liste_perturbation
        
        derniere_requete = arrivees[-1]["stop_date_time"]["base_arrival_date_time"]
    
    print(f"Dernière requete pour la gare {code_gare} à la date {derniere_requete} sur le reseau {code_reseau}")
    
    return liste_arrivee,liste_perturbation


In [10]:
def liste_id(nom_fichier):
    '''Cette fonction ouvre un fichier json et récupère une liste de toutes les clé d'un dictionnaire '''

    #Ouverture du fichier csv
    with open(nom_fichier,"r") as jsonfil:
        data_gare = json.load(jsonfil)
    toutes_id = data_gare["id"]
    liste_cles_ranked = []
    # J'ajoute toutes les clés du dictionnaire id dans une liste
    [liste_cles_ranked.append(cle) for cle in toutes_id.keys()]
    return data_gare,liste_cles_ranked

In [11]:
def run(data_gare,cle,date_min,date_max):
    '''Cette fontion lance les fonction pour traiter les donnée d'une gare'''
    code_gare = data_gare["id"].get(cle)
    print(f"Debut des requetes pour la gare: {code_gare}")

    liste_code_reseau = []
    liste_arrivees = []
    liste_perturbations = []

    #Récupération des identifiant réseaux pour la gare en cours
    [liste_code_reseau.append(reseau["id"]) for reseau in data_gare["networks"].get(cle)]

    # Je traite réseau par réseau
    for reseau in liste_code_reseau:
        liste_arrivees, liste_perturbations = requete_entre_dates(code_gare,reseau,date_min,date_max,liste_arrivees,liste_perturbations)
    
    nom_fichier_arrivees = f"data/arrivees/{code_gare}-{date_max}.json".replace(":","_")
    nom_fichier_perturbation = f"data/perturbations/{code_gare}-{date_max}.json".replace(":","_")
    to_json(liste_arrivees,nom_fichier_arrivees)
    to_json(liste_perturbations,nom_fichier_perturbation)

In [12]:
date_min = convertir_en_string(hier_debut_journee)
date_max = convertir_en_string(hier_fin_journee)

In [13]:
data_gare,liste_cle = liste_id("data/_top200gare.json")

In [ ]:
for cle in liste_cle[2:5]:
    run(data_gare,cle,date_min,date_max)